In [1]:
import cv2
import numpy as np
import os

# Path to dataset
dataset_path = "dataset"
fixed_size = (200, 200)  # Ensure all images have the same size

images = []
labels = []
label_dict = {}  # Store label-to-name mapping
current_label = 0

# Load dataset
for person_name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_name)
    if os.path.isdir(person_path):
        label_dict[current_label] = person_name
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Convert to grayscale
            
            if img is None:
                print(f"Skipping {img_path}, cannot read image.")
                continue
            
            img = cv2.resize(img, fixed_size)  # Resize to fixed size
            images.append(img)
            labels.append(current_label)
        current_label += 1

# Convert lists to numpy arrays
labels = np.array(labels)

# Train Fisherfaces Recognizer
recognizer = cv2.face.FisherFaceRecognizer_create()
recognizer.train(images, labels)

# Save the trained model
recognizer.save("fisherfaces_model.yml")

# Save label mappings
np.save("labels.npy", label_dict)

print("Training complete. Model and labels saved.")


Training complete. Model and labels saved.


In [4]:
import cv2
import numpy as np

# Load the trained Fisherfaces model
recognizer = cv2.face.FisherFaceRecognizer_create()
recognizer.read("fisherfaces_model.yml")

# Load label dictionary
label_dict = np.load("labels.npy", allow_pickle=True).item()

# Initialize face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert frame to grayscale
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_roi = gray[y:y + h, x:x + w]  # Extract face region
        face_roi = cv2.resize(face_roi, (200, 200))  # Resize for model compatibility

        # Recognize face
        label, confidence = recognizer.predict(face_roi)
        name = label_dict.get(label, "Unknown")

        # Display results
        cv2.putText(frame, f"{name} ({confidence:.2f})", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Fisherfaces Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Press 'q' to exit
        break

cap.release()
cv2.destroyAllWindows()
